<a href="https://colab.research.google.com/github/kdemertzis/TS_demer/blob/main/Drones.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install pygame

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.7/13.7 MB 56.1 MB/s eta 0:00:00


In [9]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pygame
import threading
import time

pygame.mixer.pre_init(44100, -16, 2, 512)

# Initialize Pygame
#pygame.mixer.init()

# Load the sound effect
#drone_sound = pygame.mixer.Sound('drone_alert.wav')

# Load the pre-trained model
base_model = keras.applications.MobileNetV2(
    weights='imagenet', 
    input_shape=(224, 224, 3),
    include_top=False
)

# Freeze the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

# Add classification layers
inputs = keras.Input(shape=(224, 224, 3))
x = base_model(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
outputs = layers.Dense(2, activation='softmax')(x)
model = keras.Model(inputs, outputs)

# Load the pre-trained weights
model.load_weights('model_weights.h5')

# Define the class labels
class_labels = ['not drone', 'drone']

# Define the video writer for saving the video footage
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
video_writer = cv2.VideoWriter('output.mp4', fourcc, 30.0, (640, 480))

# Define the bounding box color and thickness
bbox_color = (0, 255, 0)
bbox_thickness = 2

# Define the alert threshold and cooldown time
alert_threshold = 0.8
alert_cooldown = 5.0
last_alert_time = 0.0

# Define a flag for indicating whether a drone is detected
drone_detected = False

# Define the object tracker
tracker = cv2.TrackerKCF_create()

# Define the function for capturing and processing frames
def capture_frames(camera_id):
    global drone_detected
    cap = cv2.VideoCapture(camera_id)
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    frame_rate = int(cap.get(cv2.CAP_PROP_FPS))
    video_writer = cv2.VideoWriter(f'output{camera_id}.mp4', fourcc, frame_rate, (frame_width, frame_height))
    tracker_initialized = False
    while True:
        ret, frame = cap.read()
        if ret:
            # Resize the frame for faster processing
            small_frame = cv2.resize(frame, (224, 224))

            # Preprocess the image
            image = np.expand_dims(small_frame, axis=0)
            image = image / 255.0

            # Make a prediction
            predictions = model.predict(image)
            class_index = np.argmax(predictions)

            # Draw the class label and bounding box on the frame
            label = class_labels[class_index]
            if class_index == 1:
                drone_detected = True
                if time.time() - last_alert_time > alert_cooldown:
                    # Play the alert sound effect
                    drone_sound.play()
                    last_alert_time = time.time()
                    # Initialize the tracker
                    tracker_initialized = False
            else:
                drone_detected = False
                tracker_initialized = False
            cv2.putText(frame, label, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
            if drone_detected:
                if not tracker_initialized:
                    # Initialize the tracker with the current frame
                    bbox = (0, 0, frame_width, frame_height)
                    tracker.init(frame, bbox)
                    tracker_initialized = True
            # Update the tracker with the current frame
            success, bbox = tracker.update(frame)
            if success:
                bbox = tuple(map(int, bbox))
                cv2.rectangle(frame, bbox[:2], (bbox[0]+bbox[2], bbox[1]+bbox[3]), bbox_color, bbox_thickness)

        # Write the frame to the video writer
        video_writer.write(frame)

        # Display the frame
        cv2.imshow(f'Camera {camera_id}', frame)

        # Check for exit key
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break

# Release the resources
cap.release()
video_writer.release()
cv2.destroyAllWindows()

# Define a function for displaying the alert message
def display_alert():
  global drone_detected
  alert_font = pygame.font.SysFont('comicsansms', 72)
  while True:
    if drone_detected:

# Display the alert message on the screen
      alert_frame = np.zeros((100, 640, 3), np.uint8)
      text = alert_font.render('DRONE DETECTED!', True, (255, 0, 0))
      text_width = text.get_width()
      text_height = text.get_height()
      x = (640 - text_width) // 2
      y = (100 - text_height) // 2
      alert_frame[y:y+text_height, x:x+text_width] = text.get_rect()
      cv2.imshow('Alert', alert_frame)
    else:
      cv2.destroyWindow('Alert')
      time.sleep(0.1)

# Start the threads
      frame_threads = []
      for camera_id in range(4):
        t = threading.Thread(target=capture_frames, args=(camera_id,))
        frame_threads.append(t)
        t.start()
        alert_thread = threading.Thread(target=display_alert)
        alert_thread.start()
                   


9406464/9406464 [==============================] - 0s 0us/step


OSError: ignored